In [1]:
import sys
sys.path.insert(0, "..")
from utils import linalg
from importlib import reload
import distributed
from distributed import kernel
reload(distributed.sharded_matrix)
from distributed.sharded_matrix import ShardedMatrix, ShardedSymmetricMatrix
from distributed import matmul
import numpy as np
import pywren
from utils import misc
import math
import concurrent.futures as fs
import time
import os
from numba import jit
import scipy.linalg
from sklearn import metrics
from scipy.linalg import LinAlgError

from distributed import bcd
reload(bcd)

<module 'distributed.bcd' from '../distributed/bcd.py'>

In [2]:
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')

In [9]:
X = np.load("/home/ubuntu/imagenet-data/X_test_lift_3_3_pool_12_12_patch_stride_1.npy")


In [10]:
idxs = np.load("/home/ubuntu/imagenet-data/test_scramble_idxs.npy")


In [11]:
%time X_scrambled = X[idxs]

CPU times: user 0 ns, sys: 1.81 s, total: 1.81 s
Wall time: 1.81 s


In [12]:
X_scrambled = X_scrambled.reshape(X_scrambled.shape[0], -1)

In [13]:
%time X_block_sharded = ShardedMatrix("X_test_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_05.04.2017",\
                                shard_size_0=4096,\
                                bucket="vaishaalpywrenlinalg",\
                                reshard=True, \
                                data=X_scrambled, \
                                n_jobs=32)

Sharding matrix..... of shape (50000, 18432)
81920  122884096 016384  12288 4096204802048018432 0  
16384 24576 491520  18432 8192 500000
0 18432   024576
018432 18432 18432 
28672

 018432 
18432
36864 4096028672 4096032768 32768 45056 45056 0 368640 0   18432 018432
18432 
18432
49152 0 18432

CPU times: user 50.5 s, sys: 33.4 s, total: 1min 23s
Wall time: 36.1 s


In [12]:
X_block_sharded.get_block(0,0) - X_scrambled[:4096, :]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [9]:
pwex = pywren.default_executor(job_max_runtime=600)

In [10]:
key = "rbf(XXT(X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_blocks_no_normalize_04.22.2017, {0})".format(GAMMA)
if (not K_EXISTS):
   K = kernel.compute_kernel_pywren(pwex, key, X_block_sharded, 4, gamma=GAMMA, num_jobs=4500, num_features=X_block_sharded.shape[1])
else: 
    K = ShardedSymmetricMatrix(key, bucket="vaishaalpywrenlinalg")

Submitting jobs, chunk 0
Submitting jobs, chunk 1
Submitting jobs, chunk 2
Waiting for jobs


TypeError: compute_rbf_kernel_blockwise() missing 2 required positional arguments: 'gamma' and 'num_features'

In [48]:
K.get_block(0,0)

array([[ 0.99999988,  0.91322291,  0.90441197, ...,  0.90901983,
         0.79785383,  0.75657988],
       [ 0.91322291,  1.        ,  0.95678014, ...,  0.94905877,
         0.80784559,  0.77357572],
       [ 0.90441197,  0.95678014,  1.        , ...,  0.94054091,
         0.81784886,  0.76312709],
       ..., 
       [ 0.90901983,  0.94905877,  0.94054091, ...,  1.        ,
         0.89513743,  0.85599107],
       [ 0.79785383,  0.80784559,  0.8178488 , ...,  0.89513743,
         0.99999994,  0.93026906],
       [ 0.75657988,  0.77357572,  0.76312709, ...,  0.85599107,
         0.93026906,  1.        ]], dtype=float32)

In [11]:
idxs = np.load("/home/ubuntu/imagenet-data/scramble_idxs.npy")
labelsTrain = np.load("/home/ubuntu/imagenet-data/imagenet_train_labels.npy")[idxs]

In [12]:
y_train = np.eye(1000)[labelsTrain.astype('int')]

In [49]:
K.get_block(0,0)

array([[ 0.99999988,  0.91322291,  0.90441197, ...,  0.90901983,
         0.79785383,  0.75657988],
       [ 0.91322291,  1.        ,  0.95678014, ...,  0.94905877,
         0.80784559,  0.77357572],
       [ 0.90441197,  0.95678014,  1.        , ...,  0.94054091,
         0.81784886,  0.76312709],
       ..., 
       [ 0.90901983,  0.94905877,  0.94054091, ...,  1.        ,
         0.89513743,  0.85599107],
       [ 0.79785383,  0.80784559,  0.8178488 , ...,  0.89513743,
         0.99999994,  0.93026906],
       [ 0.75657988,  0.77357572,  0.76312709, ...,  0.85599107,
         0.93026906,  1.        ]], dtype=float32)

In [73]:
reload(bcd)

<module 'distributed.bcd' from '../distributed/bcd.py'>

In [ ]:
sys.stdout = terminal_stdout
reload(bcd)
X_1_epoch = bcd.block_kernel_solve(K, y_train, epochs=1, lambdav=1e-5, blocks_per_iter=4)

In [164]:
svd_stuff = np.linalg.svd(X_1_epoch)

In [172]:
svd_stuff[1][0]/svd_stuff[1][-1]

10493230837679216.0

In [177]:
np.diag(X_1_epoch)

memmap([ 0.38845128,  0.89646912,  0.44712415, ...,  0.26498708,
        0.56446284,  0.66869539])

In [179]:
K_block = K.get_block(0,0)

In [187]:
np.diag(K_block)

array([ 0.99999881,  0.99999988,  0.99999988, ...,  1.        ,
        0.99999952,  1.        ], dtype=float32)

In [183]:
svd_stuff[1]

array([  2.17837646e+03,   3.11883759e+02,   1.85900482e+02, ...,
         2.88411011e-05,   2.49138957e-05,   1.49347890e-17], dtype=float32)

In [42]:
x_start, y_hat =  X_1_epoch

In [145]:
%time a = bcd.fast_kernel_column_block_async(K, [0,1,2])
%time fs.wait(a)

[0, 1, 2]
CPU times: user 276 ms, sys: 1.78 s, total: 2.06 s
Wall time: 2.04 s
CPU times: user 912 ms, sys: 84 ms, total: 996 ms
Wall time: 1min 45s


DoneAndNotDoneFutures(done={<Future at 0x7f2d00221898 state=finished returned tuple>, <Future at 0x7f2ce04cf898 state=finished returned tuple>, <Future at 0x7f2ce04cf0b8 state=finished returned tuple>, <Future at 0x7f2de057b8d0 state=finished returned tuple>, <Future at 0x7f2ce04cf9e8 state=finished returned tuple>, <Future at 0x7f1f5df06a20 state=finished returned tuple>, <Future at 0x7f2ce04cfa58 state=finished returned tuple>, <Future at 0x7f2d00221a58 state=finished returned tuple>, <Future at 0x7f1f5d3a72b0 state=finished returned tuple>, <Future at 0x7f2cc2186b70 state=finished returned tuple>, <Future at 0x7f2dc0295470 state=finished returned tuple>, <Future at 0x7f2de057bc88 state=finished returned tuple>, <Future at 0x7f2dc02954a8 state=finished returned tuple>, <Future at 0x7f2ccba2e4e0 state=finished returned tuple>, <Future at 0x7f2cc993ad30 state=finished returned tuple>, <Future at 0x7f38c1699d68 state=finished returned tuple>, <Future at 0x7f2ce04cf588 state=finished ret

In [146]:
k = bcd.load_mmap(*a[0].result())

In [152]:
bcd.block_kernel_solve(K, 

memmap([[ 0.99999881,  0.40342984,  0.36615327, ...,  0.30506128,
         0.67470342,  0.40624243],
       [ 0.40342984,  0.99999988,  0.64286745, ...,  0.64839852,
         0.69248831,  0.80197585],
       [ 0.36615327,  0.64286745,  0.99999988, ...,  0.68896604,
         0.72726387,  0.8055684 ],
       ..., 
       [ 0.33696416,  0.68765193,  0.73388755, ...,  0.874237  ,
         0.59649545,  0.85790408],
       [ 0.35511884,  0.82535362,  0.73779732, ...,  0.74089462,
         0.64481544,  0.86099726],
       [ 0.28665397,  0.70327604,  0.67107129, ...,  0.90301561,
         0.5732618 ,  0.85399121]])

In [143]:
k

memmap([[ 0.99999881,  0.40342984,  0.36615327, ...,  0.38524207,
         0.1045282 ,  0.06145367],
       [ 0.40342984,  0.99999988,  0.64286745, ...,  0.59283113,
         0.11838143,  0.07674114],
       [ 0.36615327,  0.64286745,  0.99999988, ...,  0.54172248,
         0.13388455,  0.06698362],
       ..., 
       [ 0.33696416,  0.68765193,  0.73388755, ...,  0.69942391,
         0.14707921,  0.08063375],
       [ 0.35511884,  0.82535362,  0.73779732, ...,  0.62777525,
         0.11895563,  0.07865198],
       [ 0.28665397,  0.70327604,  0.67107129, ...,  0.68767077,
         0.13325712,  0.07482792]])